In [1]:
from electric_meter import *
from IPython.display import HTML

In [2]:
#meter = read_data('data/meter-big.csv')
meter = read_data('data/1694073294_9_c5108a4b.csv')

In [3]:
#meter = read_data('data/meter.csv')

costs, conf = compute_costs(meter)

In [4]:
costs

,no_discount,pazgas_daytime,pazgas_unlimited,pazgas_weekend,pazgas_nighttime,amisragas_unlimited,electra_power,electra_hitec,taoz1,taoz2
timeperiod,,,,,,,,,,
יולי 2023,649.849768,648.803592,617.35728,635.667585,568.184344,607.609533,617.35728,603.940935,809.261192,636.191192
אוגוסט 2023,715.420242,712.774616,679.64923,693.061676,630.986795,668.917926,679.64923,666.848796,855.545502,682.475502


In [5]:
result = style_table(costs)
display(result)

,חחי,פזגז יום,פזגז ללא הגבלה,פזגז סופ״ש,פזגז לילה,אמישראגז,אלקטרה פאואר,אלקטרה הייטק,תעו״ז חד-חודשי,תעו״ז דו-חודשי
תקופה,,,,,,,,,,
יולי 2023,649.85,648.80,617.36,635.67,568.18,607.61,617.36,603.94,809.26,636.19
אוגוסט 2023,715.42,712.77,679.65,693.06,630.99,668.92,679.65,666.85,855.55,682.48


In [8]:
if False:
    cols = [costs.columns[i] for i in [0,1,5, 6]]
    display(style_table(costs[cols].copy()))

In [ ]:
v = meter.query("timeperiod == '20230601'")

#meter.groupby('month').mean(numeric_only=True).plot()
meter.groupby('hour').mean(numeric_only=True).plot()

In [151]:
meter.iloc[1]['month'] = 77
meter['month'].map(month_to_season)

/var/folders/l5/kkx8dwdn57s3v0k59p9m7sv80007gv/T/ipykernel_9900/3461092686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meter.iloc[1]['month'] = 77


time
2023-01-30 02:00:00    Winter
2023-01-30 03:00:00    Winter
2023-01-30 04:00:00    Winter
2023-01-30 05:00:00    Winter
2023-01-30 06:00:00    Winter
                        ...  
2023-08-31 19:00:00    Summer
2023-08-31 20:00:00    Summer
2023-08-31 21:00:00    Summer
2023-08-31 22:00:00    Summer
2023-08-31 23:00:00    Summer
Freq: H, Name: month, Length: 5134, dtype: object

In [50]:
def taoz_rate_one_season(df, low_price, high_price, weekday_high_hours, weekend_high_hours):

    def rate_helper(df, selector, start_hour, end_hour):
        F = apply_filter(df[selector],
                         val=high_price,
                         filter_func=lambda x: filter_hour_and(x, start_hour, end_hour),
                         default_val=low_price)
        ret.loc[F.index] = F
    
    ret = pd.Series(np.nan, index=df.index)

    hhi_weekend = df['hhi_weekend']
    # weekdays
    rate_helper(df, ~hhi_weekend, weekday_high_hours['start'], weekday_high_hours['end'])
    # weekends
    rate_helper(df,  hhi_weekend, weekend_high_hours['start'], weekend_high_hours['end'])
    
    return ret

In [72]:
test_dates = [
    '2023-10-01 21:00',
    '2023-10-01 22:00',
    '2023-03-04 21:00',
    '2023-03-04 22:00',
    '2023-06-07 16:00',
    '2023-06-07 17:00',
    '2023-06-09 16:00',
    '2023-06-09 17:00',
    '2023-01-03 21:00',
    '2023-01-03 22:00',
    '2023-01-07 16:00',
    '2023-01-07 17:00',
]

mymeter = pd.DataFrame(data={'time':pd.to_datetime(test_dates)})

mymeter.set_index('time', inplace=True)

mymeter['date'] = mymeter.index.date
mymeter['wday'] = mymeter.index.weekday
mymeter['hour'] = mymeter.index.hour
mymeter['month'] = mymeter.index.month
mymeter['wday_name'] = mymeter.index.day_name()
F=filter_days(mymeter, ['Friday', 'Saturday'])
mymeter['hhi_weekend'] = F

In [73]:
mymeter

,date,wday,hour,month,wday_name,hhi_weekend
time,,,,,,
2023-10-01 21:00:00,2023-10-01,6,21,10,Sunday,False
2023-10-01 22:00:00,2023-10-01,6,22,10,Sunday,False
2023-03-04 21:00:00,2023-03-04,5,21,3,Saturday,True
2023-03-04 22:00:00,2023-03-04,5,22,3,Saturday,True
2023-06-07 16:00:00,2023-06-07,2,16,6,Wednesday,False
2023-06-07 17:00:00,2023-06-07,2,17,6,Wednesday,False
2023-06-09 16:00:00,2023-06-09,4,16,6,Friday,True
2023-06-09 17:00:00,2023-06-09,4,17,6,Friday,True
2023-01-03 21:00:00,2023-01-03,1,21,1,Tuesday,False


In [74]:
mymeter['season'] = mymeter['month'].map(month_to_season)#.unique()

In [75]:
taoz_schedule = {
    'summer' : {'low_price':'lowsummer', 'high_price':'highsummer', 'weekday_high_hours':{'start':17, 'end':23}, 'weekend_high_hours':{'start':-1, 'end':-1}},
    'winter' : {'low_price':'lowwinter', 'high_price':'highwinter', 'weekday_high_hours':{'start':17, 'end':22}, 'weekend_high_hours':{'start':17, 'end':22}},
    'transition' : {'low_price':'lowtrans', 'high_price':'hightrans', 'weekday_high_hours':{'start':17, 'end':22}, 'weekend_high_hours':{'start':-1, 'end':-1}},
}

In [76]:
for season in mymeter['season'].unique():
    season_conf=taoz_schedule[season]
    low_price = season_conf['low_price']
    high_price = season_conf['high_price']
    weekday_high_hours = season_conf['weekday_high_hours']
    weekend_high_hours = season_conf['weekend_high_hours']
    R = taoz_rate_one_season(mymeter[mymeter['season']==season], low_price, high_price, weekday_high_hours, weekend_high_hours)
    mymeter.loc[R.index, 'price'] = R

In [233]:
import pandas as pd
from pyluach.dates import HebrewDate
from pyluach import dates
from datetime import timedelta

# Sample DataFrame with datetime index
index = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=index)
df['hhi_weekend'] = False

# List of Hebrew holidays
holidays = [
    "Rosh Hashana",
    "Yom Kippur",
    "Succos",
    "Simhat Torah",
    "Pesach",
    "Yom haatzmaut",
    "Shavuos",
]

# go over the dates
holiday_dates = []
for d in df.index:
    if is_yom_atzmaut(d.year, d.month, d.day):
        holiday_dates.append(d.date())
    else:
        hd=dates.GregorianDate(d.year, d.month, d.day)
        holiday = hhi_holiday(hd)
        if holiday is not None:
            print(holiday + str(hd))
            holiday_dates.append(hd.to_pydate())



Pesach2023-4-6
Pesach2023-4-12
atmau2023-04-26 00:00:00
Shavuos2023-5-26
Rosh Hashana2023-9-16
Rosh Hashana2023-9-17
Yom Kippur2023-9-25
Succos2023-9-30
Simhat Torah2023-10-7


In [317]:
from functools import lru_cache

@lru_cache
def yom_atzmaut_heb_date(heb_year):
    target_date = dates.HebrewDate(heb_year, 2, 5)
    weekday = int(target_date.to_pydate().strftime('%w'))  # 0=Sunday
    # offsets - from https://he.wikipedia.org/wiki/%D7%99%D7%95%D7%9D_%D7%94%D7%A2%D7%A6%D7%9E%D7%90%D7%95%D7%AA
    if weekday == 5:     #Friday
        target_date -= 1
    elif weekday == 6:   # Saturday
        target_date -= 2
    elif weekday == 1:   # Monday
        target_date += 1
    return target_date.to_pydate()
    
def is_yom_atzmaut(year, month, day):
    heb_date=dates.GregorianDate(year, month, day).to_heb()
    heb_year = heb_date.year
    yom_atzmaut_date = yom_atzmaut_heb_date(heb_year)
    return yom_atzmaut_date == heb_date.to_pydate()  

def get_holidays(df):
    holiday_dates = []
    for d in set(df.index.date):
        if is_yom_atzmaut(d.year, d.month, d.day):
            holiday_dates.append(d)
        else:
            hd=dates.GregorianDate(d.year, d.month, d.day)
            holiday = hhi_holiday(hd)
            if holiday is not None:
                holiday_dates.append(hd.to_pydate())
    return holiday_dates

def set_hhi_weekends(df):
    # Get the list of holidays
    holidays = get_holidays(df)
    
    # Add the dates which precede each holiday by one day
    expanded_holidays = [date - pd.Timedelta(days=1) for date in holidays] + holidays
    
    # Add Fridays and Saturdays
    for d in set(df.index.date):
        if d.weekday() in [4,5]:
            expanded_holidays.append(d)

    # Convert the datetime index to a list of datetime.date objects
    date_list = [date.date() for date in df.index]

    df['hhi_weekend'] = [d in expanded_holidays for d in date_list]
    
    return df


In [196]:
from functools import lru_cache

@lru_cache
def yom_atzmaut_heb_date(heb_year):
    target_date = dates.HebrewDate(heb_year, 2, 5)
    weekday = int(target_date.to_pydate().strftime('%w'))  # 0=Sunday
    # offsets - from https://he.wikipedia.org/wiki/%D7%99%D7%95%D7%9D_%D7%94%D7%A2%D7%A6%D7%9E%D7%90%D7%95%D7%AA
    if weekday == 5:     #Friday
        target_date -= 1
    elif weekday == 6:   # Saturday
        target_date -= 2
    elif weekday == 1:   # Monday
        target_date += 1
    return target_date.to_pydate()
    
def is_yom_atzmaut(year, month, day):
    heb_date=dates.GregorianDate(year, month, day).to_heb()
    heb_year = heb_date.year
    yom_atzmaut_date = yom_atzmaut_heb_date(heb_year)
    return yom_atzmaut_date == heb_date.to_pydate()
    
    


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create a pivot table for heatmap
heatmap_data = meter.pivot_table(index="wday", columns="hour", values="consumption", aggfunc="sum")

# Create the heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=False, cmap="YlGnBu", fmt="")
plt.title("Consumption Heatmap")
plt.xlabel("Hour")
plt.ylabel("Month")
plt.show()
